```python
#!/usr/bin/env python
# coding: utf-8 

#   This software component is licensed by ST under BSD 3-Clause license,
#   the "License"; You may not use this file except in compliance with the
#   License. You may obtain a copy of the License at:
#                        https://opensource.org/licenses/BSD-3-Clause
  

'''
Training script of human activity recognition system (HAR), based on two different Convolutional Neural Network (CNN) architectures 
'''
```

# Step by Step HAR Training using Artificial Networks and STM32CubeAI
This notebook provides a step by step demonstration of a simple <u>H</u>uman <u>A</u>ctivity <u>R</u>ecognition system (HAR), based on a <u>C</u>onvolutional <u>N</u>eural <u>N</u>etworks (CNN). This Notebook uses a simple data preperation script through `DataHelper` class and let user to preprocess, split, and segment the dataset to bring it into the form which can be used for training and validation of the HAR CNN. For CNNs it uses a `CNNHandler` class which builds, trains and validates a CNN for a given set of input and output tensors. The `CNNHandler` comes with possibility to use one of the two provided example CNN architectures namely, **IGN** and **GMP**.

All the implementations are done in [Python](https://www.python.org/) using [Tensorflow](https://www.tensorflow.org/).

For demonstration purposes this script uses two datasets created for HAR using accelerometer sensor. 

* A public dataset provided by <u>WI</u>reless <u>S</u>ensing <u>D</u>ata <u>M</u>ining group named as **<u>WISDM</u>**. The details of the dataset are available [here](http://www.cis.fordham.edu/wisdm/dataset.php).

* Our own propritery dataset called **<u>AST</u>**. 

**Note**: We are not providing any dataset in the function pack. The user can download WISDM dataset from [here](http://www.cis.fordham.edu/wisdm/dataset.php), while **<u>AST</u>** is a private dataset and is not provided. Although a subset of the **<u>AST</u>** dataset is provided in the function pack at location `/FP-AI-MONITOR1/Utilities/AI_Resources/Datasets/AST/`.

Following figure shows the detailed workflow of CNN based HAR.


<p align="center">
<img width="760" height="400" src="workflow_nn.png">    
</p>

Now, let us implement it step by step.

### Step1 : Import necessary dependencies
Following section imports all the required dependencies. This also sets seeds for random number generators in Numpy and Tensorflow environments to make the results more deterministic between different runs.

In [ ]:
import numpy as np, tensorflow as tf, os, logging, warnings
from os.path import join
from datetime import datetime

# private libraries
from PrepareDataset import DataHelper
from HARNN import ANNModelHandler

from sklearn.metrics import accuracy_score

# for using callbacks to save the model during training and comparing the results at every epoch
from keras.callbacks import ModelCheckpoint

# disabling annoying warnings originating from Tensorflow
logging.getLogger('tensorflow').disabled = True
# disabling annoying warnings originating from python
warnings.simplefilter("ignore")

# setting the seeds to the random generators of Numpy and Tensorflow
np.random.seed( 611 )
tf.random.set_seed( 611 )

### Step2: Set environment variables
Following section sets some user variables which will later be used for:

* preparing the dataset,
* building the neural networks,
* training the neural networks, and
* validating the neural network.

In [ ]:
# data variables
dataset = 'AST' # or 'WISDM'
reducedClasses = True # or False
'''
reducedClasses = True 
    For WISDM = > 
        will merge 'Sitting' and 'Standing' classes to Stationary, and 
        'Upstairs' and 'Downstairs' to 'Stairs'. 
    For AST = >
        will remove 'Driving' Class '''
segmentLength = 24
stepSize = 24 # to control the overlap
'''
stepSize == segmentLength : No overlap
stepSize < segmentlenght : Overlap of (segmentLength - stepSize) sampels
stepSize > segmentlenght : Dropping ( stepSize - segmentLength ) sampels between adjacent samples
'''
preprocessing = True # or False. Rotate the samples so that the gravity points to 'z-axis' and supresses the gravity

# neural network architecture variables
modelName = 'IGN' # or 'GMP'

# training variables
trainTestSplit = 0.6
trainValidationSplit = 0.7
nEpochs = 20
learningRate = 0.0005
decay = 1e-6
batchSize = 64
verbosity = 1
nrSamplesPostValid = 2

### Step3: Result directory
Each run can have different variables depending on user settings and to compare the results of different choices, such as different segment size for the window for data, different overlap settings etc. We can save the results to compare different runs. 

Following section creates a result directory to save results for the current run. The name of the directory has following format. `Mmm_dd_yyyy_hh_mm_ss`, and example name for directory can be `Jul_20_2021_14_31_20/`. In every result directory there will a `.txt` file with information on the run and a saved `.h5` model.

In [ ]:
# if not already exist create a parent directory for results.
allRunsDir = './results_cnn/'
if not os.path.exists( allRunsDir ):
    os.mkdir( allRunsDir )
thisRunDir = '{}/{}/'.format( allRunsDir, datetime.now().strftime( "%Y_%b_%d_%H_%M_%S" ) )
os.mkdir( thisRunDir )
infoString = 'runTime : {}\nDatabase : {}\nNetwork : {}\nSeqLength : {}\nStepSize : {}\nEpochs : {}\n'.format( datetime.now().strftime("%Y-%b-%d at %H:%M:%S"), dataset, modelName, segmentLength, stepSize, nEpochs )
with open( thisRunDir + 'info.txt', 'w' ) as text_file:
    text_file.write( infoString )

### Step4: Create a object of class `DataHelper` named `myDataHelper`
The script in the following section creates a `DataHelper` object to preprocess, segment and split the dataset as well as to create one-hot-code labeling for the outputs to make the data training and testing ready using the choices set by the user in **Step2**.

In [ ]:
myDataHelper = DataHelper( dataset = dataset, reducedClasses = reducedClasses, seqLength = segmentLength, 
                          seqStep = stepSize, preprocessing = preprocessing, trainTestSplit = trainTestSplit,
                          trainValidSplit = trainValidationSplit, resultDir = thisRunDir )

## Step5: Prepare the dataset
Following section prepares the dataset and create six tensors namely `TrainX`, `TrainY`, `ValidationX`, `ValidationY`, `TestX`, `TestY`. Each of the variables with trailing `X` are the inputs with shape `[_, segmentLength, nr_of_axes = 3, nr_channels = 1 ]`and each of the variables with trailing `Y` are corresponding outputs with shape `[ _, NrClasses ]`. `NrClasses` for `WISDM` can be `4` or `6` and for `AST` are `4` or `5` depending if the `reducedClasses` flag is set to `True` or `False`.

In [ ]:
TrainX, TrainY, ValidationX, ValidationY, TestX, TestY = myDataHelper.prepare_data()

### print the number of samples in train, test and validation data sets and the number of classes

In [ ]:
print( 'Number of training samples : {}\nNumber of validation samples : {}\nNumber of test samples : {}\nNumber of classes : {}'.\
      format( TrainX.shape[0], ValidationX.shape[0], TestX.shape[0], TrainY.shape[1] ) )

### Step6: Create an object of classe `ANNModelHandler`
The script in the following section creates a `ANNModelHandler` object to create, train and validate the <u>C</u>onvolutional <u>N</u>eural <u>N</u>etwork (**CNN**) using the variables created in **Step2**.

In [ ]:
myHarHandler = ANNModelHandler( modelName = modelName, classes = myDataHelper.classes, resultDir = thisRunDir,
                              inputShape = TrainX.shape, outputShape = TrainY.shape, learningRate = learningRate,
                              decayRate = decay, nEpochs = nEpochs, batchSize = batchSize,
                              modelFileName = 'har_' + modelName, verbosity = verbosity )

#### Step6.1: Create a CNN model
Following script creates the **<u>CNN</u>** and prints its summary to show the architecture and provide the information on the trainable parameters.

In [ ]:
harModel = myHarHandler.build_model()
harModel.summary()

#### Step6.2: Create a Checkpoint for ANN training
The following script creates a check point for the training process of ANN to save the neural network as `h5` file. The settings are used in a way that the copy with maximum validation accuracy `val_acc` is saved only.

In [ ]:
harModelCheckPoint = tf.keras.callbacks.ModelCheckpoint( filepath = join( thisRunDir, 'har_' + modelName + '.h5' ),
                                     monitor = 'val_acc', verbose = 0, save_best_only = True, mode = 'max' )

### Step7 : Train the created <u>CNN</u> model
The following script trains the created **<u>CNN</u>** with the provided checkpoint and created datasets.

In [ ]:
harModel = myHarHandler.train_model( harModel, TrainX, TrainY, ValidationX, ValidationY, harModelCheckPoint )

In [ ]:
print( 'Training accuracy', 
      round( 100 * accuracy_score( np.argmax( TrainY, axis = 1 ), 
                                  np.argmax( harModel.predict( TrainX ), axis = 1 ) ), 2 ) )
print( 'Test accuracy', 
      round( 100 * accuracy_score( np.argmax( TestY, axis = 1 ), 
                                  np.argmax( harModel.predict( TestX ), axis = 1 ) ), 2 ) )
print( 'Validation accuracy', 
      round( 100 * accuracy_score( np.argmax( ValidationY, axis = 1), 
                                  np.argmax( harModel.predict( ValidationX ), axis = 1 ) ), 2 ) )

### Step8: Validating the trained <u>CNN</u> model on test data
The following section validates the trained network and creates a confusion matrix for the test dataset to have a detailed picture of the distributions of the errors.

In [ ]:
myHarHandler.make_confusion_matrix(  harModel, TrainX, TrainY )

### Step9: Validating the trained <u>CNN</u> on the data acquire using High Speed Datalogger on STWIN
Following we test the performance of the trained neural network on the data acquired using the High Speed Datalogger for each of the activities to see the how the model trained on the dataset performs on the data logged in live conditions.

#### Step9.1: Stationary activity

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_nn_segments( '../../Datasets/HSD_Logged_Data/HAR/Stationary/', 
                                                         activityName = 'Stationary' )
print( 'Validation accuracy', 
      round( 100 * accuracy_score(np.argmax(labels, axis=1), 
                                  np.argmax(harModel.predict(segments), axis =1 )),2 ), '%' )

#### Step9.2: Walking activity

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_nn_segments( '../../Datasets/HSD_Logged_Data/HAR/Walking/', 
                                                         activityName = 'Walking' )
print( 'Validation accuracy', 
      round( 100 * accuracy_score(np.argmax(labels, axis=1), 
                                  np.argmax(harModel.predict(segments), axis =1 )),2 ), '%' )

#### Step9.3: Jogging activity

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_nn_segments( '../../Datasets/HSD_Logged_Data/HAR/Jogging/', 
                                                         activityName = 'Jogging' )
print( 'Validation accuracy', 
      round( 100 * accuracy_score(np.argmax(labels, axis=1), 
                                  np.argmax(harModel.predict(segments), axis =1 )),2 ), '%' )

#### Step9.4: Biking activity
This section will only run when AST dataset is used which have `Biking` class. if WISDM dataset is used this section will generate an error as the `Biking` does not exist.

In [ ]:
segments, labels = myDataHelper.hsdatalog_to_nn_segments( '../../Datasets/HSD_Logged_Data/HAR/Biking/', 
                                                         activityName = 'Biking' )
print( 'Validation accuracy', 
      round( 100 * accuracy_score(np.argmax(labels, axis=1), 
                                  np.argmax(harModel.predict(segments), axis =1 )),2 ), '%' )

### Step10: Create an npz file for validation after conversion from CubeAI.

In [ ]:
myDataHelper.dump_data_for_post_validation( TestX, TestY, nrSamplesPostValid )

### Additional work to study the size of the dataset to obtain 85% performance
This section is written to compare the performance of the **<u>CNN</u>** vs **<u>SVC</u>**.
As the idea is that when small dataset is available we do not need the CNN for good performance instead SVC can be an easy approach to have comparable or even better performances.

In [ ]:
testX = np.concatenate( ( TestX, ValidationX ) )
testy = np.concatenate( ( TestY, ValidationY ) )
print( 'samples,train_acc,test_acc')
if( TrainY.shape[0] < 4000 ):
    print( 'not enough samples present for this analysis' )
else:
    for ii in range( 100, 4000, 100 ):
        harModelTest = myHarHandler.build_model()
        harModelTest.fit( TrainX[ : ii ], TrainY[ : ii ], 
                 validation_split = 0.3, batch_size = 64, epochs = 30, verbose = 0 )
        trainAcc = round( 100 * accuracy_score(np.argmax( TrainY, axis = 1 ), 
                                           np.argmax( harModelTest.predict( TrainX ), axis = 1 ) ), 2 ) 
        testAcc = round( 100 * accuracy_score( np.argmax( testy, axis = 1 ), 
                                          np.argmax( harModelTest.predict( testX ), axis = 1 ) ), 2 ) 
        print( '{},{},{}'.format( ii, trainAcc, testAcc ) )


```python
### code to generate the graph below
import pandas as pd
data = pd.read_csv('./svm_vs_nn.txt', delimiter = ',', dtype = 'a' )
fig = plt.figure(figsize = (20,10))
plt.plot( data['samples'].values.astype(np.float32), data['svm_tt'].values.astype(np.float32), linewidth = 5 )
plt.plot( data['samples'].values.astype(np.float32), data['nn_tt'].values.astype(np.float32), linewidth = 5 )
plt.legend([ 'SVC', 'CNN' ], fontsize = 25)
plt.xlabel('Training Samples', fontsize = 25)
plt.ylabel('Accuracy [%]', fontsize = 25)
plt.title('SVC vs CNN', fontsize = 35)
plt.xticks(range(500, 4000, 500 ), fontsize = 25)
plt.yticks(range(50, 110, 10 ), fontsize = 25)
plt.grid()
plt.show()
plt.savefig('svc_vs_cnn_acc.jpg')
```
<p align="center">
<img width="900" height="400" src="svc_vs_cnn_acc.jpg">    
</p>